# Database

#### **Athlete Biography**

| Column | Type (value) | Pre-process |
| :-: | :-: | :-- |
| athlete_id      | int                ||
| name            | str                ||
| sex             | str (Male/Female)  ||
| born            | str (date)         | Standarize format (datetime, default 01/01/YYYY) |
| height          | int (cm)           ||
| weight          | int (kg)           ||
| country         | str (country_name) ||
| country_noc     | str (country_noc)  ||
| description     | str (info)         ||
| spetial_notes   | str (info)         ||

#### **Athlete Event Details**

| Column | Type (value) | Pre-process |
| :-: | :-: | :-- |
| edition     | str (year + event)      ||
| edition_id  | int                     ||
| country_noc | str (of the athlete)    ||
| sport       | str (of the athlete)    ||
| event       | str (within the sport)  ||
| result_id   | int (results)           ||
| athlete     | str (name)              ||


#### **Country Profiles**

| Column | Type (value) | Pre-process |
| :-: | :-: | :-- |
| noc       | str (3-letter word)   ||
| country   | str (name)            | Remove ROC |

#### **Event Results**

| Column | Type (value) | Pre-process |
| :-: | :-: | :-- |
| result_id             | int                               ||
| event_title           | str (title, sex)                  ||
| edition               | str (year + event)                ||
| edition_id            | int                               ||
| sport                 | str (of the event)                ||
| sport_url             | web (/editions/DD/sports/SSS)     ||
| result_date           | str (date + time)                 | Standarize format (datetime) |
| result_location       | str (loc info)                    | Standarize format (city, country) |
| result_participants   | str (number of people)            ||
| result_format         | str (description on how to win)   ||

#### **Games Summary**

| Column | Type (value) | Pre-process |
| :-: | :-: | :-- |
| edition           | str (year + event)    ||
| edition_id        | int                   ||
| edition_url       | web (/editiond/DD)    ||
| year              | int                   ||
| city              | str (name)            ||
| country_flag_url  | url (https://...)     ||
| country_noc       | str (3-letter word)   ||
| start date        | str (datetime)        | Standarize format (datetime)
| end_date          | str (datetime)        | Standarize format (datetime)
| competition_date  | str (datetimes)       | Standarize format (datetime)

#### **Medal Tally History**

| Column | Type (value) | Pre-process |
| :-: | :-: | :-- |
| edition           | str (year + event)    ||
| edition_id        | int                   ||
| year              | int                   ||
| country           | str (name)            ||
| country_noc       | str (3-letter word)   ||
| gold              | int                   ||
| silver            | int                   ||
| bronze            | int                   ||
| total             | int (sum of previous) ||

# Code

In [ ]:
# Imports

import pandas as pd
import numpy as np
from datetime import date
import country_converter as coco

cc = coco.CountryConverter()

In [ ]:
# Reading in Data

class OlympicDatabase():
    def __init__(self):
        try:
            self.df_athlete_biography       = pd.read_csv('./raw_data/Olympic_Athlete_Biography.csv')
            self.df_athlete_event_details   = pd.read_csv('./raw_data/Olympic_Athlete_Event_details.csv')
            self.df_country_profiles        = pd.read_csv('./raw_data/Olympic_Country_Profiles.csv')
            self.df_event_results           = pd.read_csv('./raw_data/Olympic_Event_Results.csv')
            self.df_games_summary           = pd.read_csv('./raw_data/Olympic_Games_Summary.csv')
            self.df_medal_tally             = pd.read_csv('./raw_data/Olympic_Medal_Tally_History.csv')
        except FileNotFoundError as error:
            print('File Not Found: ' + error.filename)
        else:
            self.df_dict = {"athlete_biography":     self.df_athlete_biography,
                            "athlete_event_details": self.df_athlete_event_details,
                            "country_profiles":      self.df_country_profiles,
                            "event_results":         self.df_event_results,
                            "games_summary":         self.df_games_summary,
                            "medal_tally":           self.df_medal_tally}
            self._preprocess()
    
    def _preprocess(self):
        # Preprocess born data from string to datetime ->  YYYY-MM-DD
        self.df_athlete_biography['born'] = self.df_athlete_biography['born'].apply(self.__find_date)

    def __find_date(self, txt):
        txt = str(txt)   # In case txt = 1920 (numeric)
        numbers = '0123456789'
        months  = ['january', 'february', 'march', 'april', 'may', 'june',
                'july', 'august', 'september', 'october', 'november', 'december']
        text    = list(txt)

        day = None
        month = None
        year = None

        # Find day and year
        prev_carac = ' '
        for i, carac in enumerate(text):
            if carac in numbers and prev_carac not in numbers: # Check for starting number
                try:
                    if text[i+1] in numbers:                # 12...
                        if text[i+2] not in numbers:        # 12 ...
                            day = int(''.join(text[i:i+2]))
                        elif text[i+3] in numbers:          # 1234...
                            year = int(''.join(text[i:i+4]))     # Keeps last year: "(1920 or 1921)" -> 1921
                    else:                                   # 1 ...
                        day = int(carac)
                except:
                    continue
            prev_carac = carac
        # Simple check for month
        for word in txt.split(' '):
            if word.lower() in months:
                month = months.index(word.lower())+1
        
        # Without year, there is no way of telling
        if year is None:
            return np.nan
        
        # Update incompleted dates:
        if day is None:
            day = 1
        if month is None:
            month = 1
        
        # Return representative values
        return date(year, month, day).isoformat()

    def process(self, df_name, col_name, **handler):
        df_name = df_name.lower().replace(" ","_")  # "Medal Tally -> medal_tally"
        data_name = None
        for name in self.df_dict.keys():
            if df_name in name:
                try:
                    data_name = name
                except KeyError:
                    print("Error: Column not found in given database")
                    return
        if data_name is None:
            print("Error: Databse not found")
            return
        for key, value in handler.items():
            if key == "replace_all":
                print("Replacing All NaN values to", value)
                self.df_dict[data_name][col_name] = self.df_dict[data_name][col_name].fillna(value)
        # Changes the internal dataframe

data = OlympicDatabase()

In [4]:
def world_map_data(**kwargs):
    # in:      None. ¿Sport and Event selection?
    # out:     list(tuple(id, number of total medals))
    # default: All history

    # default algorithm (no filters, all history)
    data.process(df_name= "medal_tally", col_name= "total", replace_all= 0)
    grouped    = data.df_medal_tally.groupby("country")["total"].sum()  # Sum of all medals for each country
    iso2_codes = cc.convert(names= grouped.keys(), to= "ISO2")
    # Not found is not considered for World Map
    return list(zip(iso2_codes, map(int, grouped.values)))

print(world_map_data())


    
    
    

def list_of_sports(country_noc):
    # in:      str(Country_noc) | any identifyer of the country, to be defined
    # out:     list(Sports)     | Sports that the country have competed in
    # default: all sports
    pass

def ditr_corr(x, sport, *years):
    # in:      x, sport, year(s)
    # out:     age, weight, height
    # default: None
    pass

def get_age(athlete_id):
    pass

Australasia not found in regex
East Germany not found in regex
Independent Olympic Athletes not found in regex
Mixed team not found in regex
Netherlands Antilles not found in regex
ROC not found in ISO3
Serbia and Montenegro not found in regex
Soviet Union not found in regex
Unified Team not found in regex
United Arab Republic not found in regex
West Germany not found in regex
West Indies Federation not found in regex
Yugoslavia not found in regex


Replacing All NaN values to 0
[('AF', 2), ('DZ', 17), ('AR', 77), ('AM', 18), ('not found', 12), ('AU', 563), ('AT', 361), ('AZ', 49), ('BH', 4), ('BB', 1), ('BY', 105), ('BE', 181), ('BM', 2), ('CZ', 6), ('BW', 2), ('BR', 150), ('BG', 230), ('BF', 1), ('BI', 2), ('CM', 6), ('CA', 553), ('CL', 13), ('TW', 36), ('CO', 34), ('CR', 4), ('HR', 52), ('CU', 235), ('CY', 1), ('CZ', 101), ('CZ', 171), ('CI', 4), ('KP', 57), ('DK', 221), ('DJ', 1), ('DO', 12), ('not found', 519), ('EC', 5), ('EG', 36), ('ER', 1), ('EE', 44), ('ET', 58), ('FJ', 3), ('FI', 489), ('FR', 951), ('GA', 1), ('GE', 40), ('DE', 1098), ('GH', 5), ('GB', 988), ('GR', 157), ('GD', 3), ('GT', 1), ('GY', 1), ('HT', 2), ('HK', 9), ('HU', 535), ('IS', 4), ('not found', 5), ('IN', 35), ('ID', 37), ('IQ', 1), ('IE', 38), ('IR', 76), ('IL', 13), ('IT', 789), ('JM', 88), ('JP', 575), ('JO', 3), ('KZ', 79), ('KE', 113), ('SA', 4), ('XK', 3), ('KW', 3), ('KG', 7), ('LV', 31), ('LB', 4), ('LI', 10), ('LT', 26), ('LU', 7), ('MY', 13),

In [5]:
print(data.df_athlete_event_details.shape)

df_filtered = data.df_athlete_event_details[data.df_athlete_event_details["medal"].notna()]

print(df_filtered[df_filtered["medal"] == "Bronze"].shape)
print(df_filtered[df_filtered["medal"] == "Silver"].shape)
print(df_filtered[df_filtered["medal"] == "Gold"].shape)

# Some rows dont match but we dont really care we are gonna rely on medals
display(df_filtered[df_filtered["pos"] == "3"][df_filtered["medal"] != "Bronze"])
display(df_filtered[df_filtered["pos"] == "2"][df_filtered["medal"] != "Silver"])
display(df_filtered[df_filtered["pos"] == "1"][df_filtered["medal"] != "Gold"])



(316834, 11)
(14939, 11)
(14676, 11)
(15072, 11)


C:\Users\pio\AppData\Local\Temp\ipykernel_32916\447431652.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(df_filtered[df_filtered["pos"] == "3"][df_filtered["medal"] != "Bronze"])


,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
284348,1912 Summer Olympics,6,SWE,Athletics,"Decathlon, Men",56970,Charles Lomberg,76283,3,Silver,False
296935,1912 Summer Olympics,6,USA,Athletics,"Pentathlon, Men",57014,Jim Donahue,78331,3,Silver,False


C:\Users\pio\AppData\Local\Temp\ipykernel_32916\447431652.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(df_filtered[df_filtered["pos"] == "2"][df_filtered["medal"] != "Silver"])


,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
31133,2016 Summer Olympics,59,EGY,Weightlifting,"Middleweight, Men",354317,Mohamed Ihab,133608,2,Bronze,False
62650,2014 Winter Olympics,58,AUT,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Christoph Sumann,101195,2,Bronze,True
62651,2014 Winter Olympics,58,AUT,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Daniel Mesotitsch,101212,2,Bronze,True
62652,2014 Winter Olympics,58,AUT,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Simon Eder,118582,2,Bronze,True
62653,2014 Winter Olympics,58,AUT,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Dominik Landertinger,118518,2,Bronze,True
68358,2012 Summer Olympics,54,KAZ,Weightlifting,"Light-Heavyweight, Women",331063,Anna Nurmukhambetova,126715,2,Bronze,False


C:\Users\pio\AppData\Local\Temp\ipykernel_32916\447431652.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(df_filtered[df_filtered["pos"] == "1"][df_filtered["medal"] != "Gold"])


,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
74641,2000 Summer Olympics,25,GRE,Athletics,"100 metres, Women",65683,Aikaterini Thanou,70693,1,Silver,False
144927,2014 Winter Olympics,58,GER,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Erik Lesser,127808,1,Silver,True
144928,2014 Winter Olympics,58,GER,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Daniel Böhm,127807,1,Silver,True
144929,2014 Winter Olympics,58,GER,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Arnd Peiffer,118509,1,Silver,True
144930,2014 Winter Olympics,58,GER,Biathlon,"4 × 7.5 kilometres Relay, Men",350026,Simon Schempp,118583,1,Silver,True
192783,2016 Summer Olympics,59,CHN,Weightlifting,"Middleweight, Men",354317,Lu Xiaojun,121984,1,Silver,False
